In [ ]:
# | default_exp _cli

In [ ]:
# | export

import importlib
import sys
import asyncio
from pathlib import Path
from typing import *
import signal
from os import getpid
import time
import anyio
import threading
import copy

import typer
from fastapi import FastAPI

from fastkafka.application import FastKafka
from fastkafka._components.logger import get_logger, supress_timestamps
from fastkafka._components.helpers import _import_from_string

In [ ]:
import os
from contextlib import contextmanager
from tempfile import TemporaryDirectory

import nbformat
from nbconvert import PythonExporter
from typer.testing import CliRunner

from fastkafka.testing import change_dir
from fastkafka._components.helpers import generate_app_src

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
runner = CliRunner()

In [ ]:
# | export

_app = typer.Typer(help="")


@_app.command(
    help="Runs Fast Kafka API application",
)
def run(
    num_workers: int = typer.Option(
        1, help="Number of FastKafka instances to run"
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    pass


@_app.command(
    help="Creates documentation for a Fast Kafka API application ",
)
def generate_docs(
    root_path: str = typer.Option(
        ".", help="root path under which documentation will be created"
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    try:
        application = _import_from_string(app)
        application.skip_docs = False
        application.create_docs()
    except Exception as e:

        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
result = runner.invoke(_app, ["--help"])

                                                                                                                   
 Usage: root [OPTIONS] COMMAND [ARGS]...                                                                           
                                                                                                                   

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --install-completion          Install completion for the current shell.                                         │
│ --show-completion             Show completion for the current shell, to copy it or customize the installation.  │
│ --help                        Show this message and exit.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Commands ──────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ generate-docs           Creates documentation for a Fast Kafka API application                                  │
│ run                     Runs Fast Kafka API application                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["generate-docs", "--help"])

                                                                                                                   
 Usage: root generate-docs [OPTIONS] APP                                                                           
                                                                                                                   

Creates documentation for a Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT  root path under which documentation will be created [default: .]                       │
│ --help                   Show this message and exit.                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with TemporaryDirectory() as d:
    src_path = Path(d) / "main.py"
    generate_app_src(src_path)
    print(f"{src_path=}")
    with change_dir(d):
        !ls -al {d}
        import_str = f"{src_path.stem}:kafka_app"

        result = runner.invoke(_app, ["generate-docs", import_str])
        typer.echo(result.output)
        assert result.exit_code == 0

        result = runner.invoke(_app, ["generate-docs", import_str])
        typer.echo(result.output)
        assert result.exit_code == 0

src_path=PosixPath('/tmp/tmp9yw4ad8x/main.py')
total 11
drwx------ 2 tvrtko tvrtko     3 Jan 31 13:01 .
drwxrwxrwt 5 root   root       8 Jan 31 13:01 ..
-rw-rw-r-- 1 tvrtko tvrtko 10829 Jan 31 13:01 main.py
[INFO] main: check
[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmp9yw4ad8x/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmp9yw4ad8x/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
[INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/tmp9yw4ad8x/asyncapi/docs.






In [ ]:
result = runner.invoke(_app, ["run", "--help"])

                                                                                                                   
 Usage: root run [OPTIONS] APP                                                                                     
                                                                                                                   

Runs Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --num-workers        INTEGER  Number of FastKafka instances to run [default: 1]                                 │
│ --help                        Show this message and exit.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
import nest_asyncio
nest_asyncio.apply()